In [195]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [196]:
driver = webdriver.Chrome(ChromeDriverManager().install())

fox = "https://cctigers.com/sports/mens-basketball/stats/2022-2023/george-fox-university/boxscore/8903"
braska = "https://cctigers.com/sports/mens-basketball/stats/2022-2023/nebraska-wesleyan-university/boxscore/8905"

driver.get(fox)
soup_fox = BeautifulSoup(driver.page_source, 'html.parser')
driver.get(braska)
soup_braska = BeautifulSoup(driver.page_source, 'html.parser')

driver.quit()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 107.0.5304
[WDM] - Get LATEST chromedriver version for 107.0.5304 google-chrome
[WDM] - Driver [/Users/jeremydumalig/.wdm/drivers/chromedriver/mac64/107.0.5304.62/chromedriver] found in cache
/var/folders/91/7lxj2xq13852sql471wdl0pw0000gn/T/ipykernel_43735/3167238115.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [197]:
half1 = soup_fox.find_all('table', {'class' : 'sidearm-table play-by-play'})[0]
half2 = soup_fox.find_all('table', {'class' : 'sidearm-table play-by-play'})[1]
half3 = soup_braska.find_all('table', {'class' : 'sidearm-table play-by-play'})[0]
half4 = soup_braska.find_all('table', {'class' : 'sidearm-table play-by-play'})[1]

In [202]:
def action_team_fox(action):
    if "by" in [i.text for i in action][0]:
        return 'Opponent'
    else:
        return 'Colorado'
    
def action_team_braska(action):
    if "by" in [i.text for i in action][0]:
        return 'Colorado'
    else:
        return 'Opponent'

In [281]:
def soup_to_df(half, opp):
    times = [i.find('td').text for i in half.find_all('tr')[1:]]

    raw_actions = [i.find_all('td', {'class' : 'hide-on-medium-down'}) for i in half.find_all('tr')[1:]]
    actions = [[i.text for i in action if 'by' in i.text][0] for action in raw_actions]

    if opp == 'Fox':
        teams = [action_team_fox(a) for a in raw_actions]
    else:
        teams = [action_team_braska(a) for a in raw_actions]

    game = pd.DataFrame({'Time' : ["20:00"] + times, 
                         'Action' : ["TIP by "] + actions, 
                         'Team' : [teams[0]] + teams})
    
    game['Action'] = game['Action'].apply(lambda s : s.split(" by ")[0])
    
    return game

df1 = soup_to_df(half1, 'Fox')
df2 = soup_to_df(half2, 'Fox')
df3 = soup_to_df(half3, 'Braska')
df4 = soup_to_df(half4, 'Braska')

In [286]:
def time_passed(start, end):
    if (start == "--") and (end == "02:34"):
        return 21
    if (start == "10:39") and (end == "--"):
        return 0
    
    start_time = int(start.split(":")[0])*60 + int(start.split(":")[1])
    end_time = int(end.split(":")[0])*60 + int(end.split(":")[1])
    
    if start_time > end_time:
        return start_time - end_time
    else:
        return 0

In [369]:
def possession_length(df):
    possessions = []
    current = 1
    for x in range(0, len(df)):
        indexer = df.shape[0] - x - 1
        if ('MISS' in df.Action.values[indexer]) | ('GOOD' in df.Action.values[indexer]) | ('TURNOVER' in df.Action.values[indexer]):
            possessions.append(current)
            current += 1
        else:
            possessions.append(current)

    df['Possession'] = possessions[::-1]

    lengths = []
    outcomes = []
    for x in pd.unique(df.Possession):
        new_df = pd.concat([df[(df.Possession == x)],
                            df[(df.index == 1 + df[(df.Possession == x)].index.values[-1])] ])

        new_df = new_df[(~new_df.Action.str.contains("SUB")) & (~new_df.Action.str.contains("TIMEOUT"))]

        if "FT" in new_df.Action.values[-1]:
            outcome = "Shooting Foul"
        elif "MISS" in new_df.Action.values[-1]:
            outcome = "Miss"
        elif "GOOD" in new_df.Action.values[-1]:
            outcome = "Make"
        elif "TURNOVER" in new_df.Action.values[-1]:
            outcome = "Turnover"
        else:
            outcome = "FLAG"

        start = new_df.Time.values[0]
        if new_df.Time.values[-1] == '--':
            end = new_df.Time.values[-2]
        else:
            end = new_df.Time.values[-1]

        if new_df.Team.values[-1] == 'Colorado':
            new_df['new'] = new_df.Action.apply(lambda i : "FT" in i)
            if (new_df.new.sum() <= 1) and (outcome != "FLAG"):
                lengths.append(time_passed(start, end))
                outcomes.append(outcome)
            
    return pd.DataFrame({'Length' : lengths, 'Outcome' : outcomes})

In [370]:
colorado = pd.concat([possession_length(df1),
                      possession_length(df2),
                      possession_length(df3),
                      possession_length(df4)])

In [371]:
colorado['<10sec'] = colorado.Length < 10
colorado['>=10sec'] = colorado.Length >= 10

In [372]:
under10 = colorado[(colorado['<10sec'])].groupby('Outcome').count()
under10['Count'] = under10['Length']
under10['%'] = 100 * round(under10.Count / sum(under10.Count), 3)

print("Possessions lasting <10 seconds:")
display(under10[['Count', '%']])



over10 = colorado[(colorado['>=10sec'])].groupby('Outcome').count()
over10['Count'] = over10['Length']
over10['%'] = 100 * round(over10.Count / sum(over10.Count), 3)

print("Possessions lasting 10+ seconds:")
display(over10[['Count', '%']])

Possessions lasting <10 seconds:


,Count,%
Outcome,,
Make,19,27.9
Miss,19,27.9
Shooting Foul,16,23.5
Turnover,14,20.6


Possessions lasting 10+ seconds:


,Count,%
Outcome,,
Make,27,24.3
Miss,47,42.3
Shooting Foul,6,5.4
Turnover,31,27.9


In [373]:
by_length = colorado.groupby("<10sec").count()

by_length['Count'] = by_length.Length
by_length['%'] = 100 * round(by_length.Count / sum(by_length.Count), 3)

by_length['Possession Length'] = by_length.index.map({False : "10+ sec", True : "<10 sec"})
by_length.index = by_length['Possession Length']

by_length[['Count', '%']].sort_values(by='Possession Length', ascending=False)

,Count,%
Possession Length,,
<10 sec,68,38.0
10+ sec,111,62.0


In [368]:
colorado.Length.mean().round(1)

14.2